In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import re
import yaml
from sariModelInterpreter import modelInterpreter as mi

In [3]:
inputFile = '../models/bso.yml'

In [4]:
model = mi.parseModelFromFile(inputFile)

In [5]:
mi.verifyModel(model)

'Ok'

In [6]:
print(mi.compileQueryForNodes(model, 
                              'artwork',
                              ['work_creator', 'work_creation_subcreation_actor', 'work_creation_subcreation_type'], 
                              group=['work_creator'], 
                              optional=['work_creation_subcreation_actor', 'work_creation_subcreation_type']))


SELECT ($subject as ?artwork) (GROUP_CONCAT(?value_work_creator) as ?work_creator) (?value_work_creation_subcreation_actor as ?work_creation_subcreation_actor) (?value_work_creation_subcreation_type as ?work_creation_subcreation_type) {
 $subject crm:P128_carries ?value_work . 
 ?value_work crm:P94i_was_created_by ?value_work_creation . 
 ?value_work_creation crm:P14_carried_out_by ?value_work_creator . 
OPTIONAL {
	 $subject crm:P128_carries ?value_work . 
	 ?value_work crm:P94i_was_created_by ?value_work_creation . 
	 ?value_work_creation crm:P9_consists_of ?value_work_creation_subcreation . 
	 ?value_work_creation_subcreation crm:P14_carried_out_by ?value_work_creation_subcreation_actor . 
	}
OPTIONAL {
	 $subject crm:P128_carries ?value_work . 
	 ?value_work crm:P94i_was_created_by ?value_work_creation . 
	 ?value_work_creation crm:P9_consists_of ?value_work_creation_subcreation . 
	 ?value_work_creation_subcreation crm:P2_has_type ?value_work_creation_subcreation_type . 
	}
} GROU